In [ ]:
import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
import ctypes
from ctypes import c_double, c_int, Structure


In [ ]:
# IMPORTANT: NEED TO INSTALL ROS RO BE ABLE TO READ THE SIMULINK MODEL FOR NOW !
# sudo apt-get update
# sudo apt-get install libroscpp-dev

In [ ]:
# Define the function prototype if necessary
# For example, if your function is named `updateVehicleState` and takes a pointer to VehicleState and a double dt
# updateVehicleState = lib.updateVehicleState
# updateVehicleState.argtypes = [ctypes.POINTER(VehicleState), c_double]
# updateVehicleState.restype = None

In [ ]:
class VehicleState(Structure):
    _fields_ = [("x", c_double),
                ("y", c_double),
                ("yaw", c_double),
                ("v_x", c_double),
                ("v_y", c_double),
                ("r", c_double),
                ("a_x", c_double),
                ("a_y", c_double),
                ("steer", c_double)]


In [30]:
import time
import ctypes
import os

# Define realT type used by the shared library
real_T = ctypes.c_double

class Vehicle:

    def __init__(self):
        
        # Add the current directory to LD_LIBRARY_PATH
        current_dir = os.getcwd()
        current_ld_library_path = os.environ.get('LD_LIBRARY_PATH', '')
        new_ld_library_path = os.path.abspath('.') + ':' + current_ld_library_path
        os.environ['LD_LIBRARY_PATH'] = new_ld_library_path

        # Construct the full path to the shared library
        libvehicle_path = os.path.join(current_dir, 'libvehicle_and_lowlevel_control.so')

        # Load the shared library
        self.lib = ctypes.CDLL(libvehicle_path)



        # Define CarModel model class
        CarModel_new = self.lib.CarModel_new
        CarModel_new.restype = ctypes.c_void_p

        # Define the initialize method of the CarModel class
        CarModel_initialize = self.lib.CarModel_initialize
        CarModel_initialize.argtypes = [ctypes.c_void_p]

        # Define the destructor of the CarModel class
        self.CarModel_delete = self.lib.CarModel_delete
        self.CarModel_delete.argtypes = [ctypes.c_void_p]
        
        # Create an instance of the car model and initialize it
        self.car_model_instance = CarModel_new()
        CarModel_initialize(self.car_model_instance)

        # Set state to 0
        self.state = {
            'x': 0.0, 'y': 0.0, 'yaw': 0.0,
            'v_x': 0.0, 'v_y': 0.0, 'r': 0.0,
            'a_x': 0.0, 'a_y': 0.0, 'steer': 0.0,
        }

        # Define the variables to be used in the step function
        self.arg_x = real_T()
        self.arg_y = real_T()
        self.arg_phi = real_T()

        self.arg_VE_vx, self.arg_VE_vy, self.arg_VE_r  = real_T(), real_T(), real_T()
        self.arg_VE_ax, self.arg_VE_ay = real_T(), real_T()

        self.arg_steering_rad, self.arg_steering_rate, self.arg_steering_torque = real_T(), real_T(), real_T()

        self.arg_DV_DC, self.arg_DV_delta, self.arg_yaw_moment_target = real_T(), real_T(), real_T()

        # Define the time step
        dt = 0.5

        self.step_func = self.lib.CarModel_step
        self.step_func.argtypes = [
            ctypes.c_void_p,  # instance pointer
            real_T, real_T,  # tuning_param_key, tuning_param_value
            real_T, real_T,  # Command_throttle, Command_steering
            real_T, real_T, real_T,  # a_x_target0, a_x_target1, a_x_target2
            real_T, real_T, real_T,  # delta_target0, delta_target1, delta_target2
            real_T, real_T, real_T,  # yawrate_target0, yawrate_target1, yawrate_target2
            real_T, real_T, real_T,  # vx_mpc0, vx_mpc1, vx_mpc2
            real_T, real_T, real_T,  # vy_mpc0, vy_mpc1, vy_mpc2
            real_T, real_T, real_T,  # F_x_target_FL_0, F_x_target_FL_1, F_x_target_FL_2
            real_T, real_T, real_T,  # F_x_target_FR_0, F_x_target_FR_1, F_x_target_FR_2
            real_T, real_T, real_T,  # F_x_target_RL_0, F_x_target_RL_1, F_x_target_RL_2
            real_T, real_T, real_T,  # F_x_target_RR_0, F_x_target_RR_1, F_x_target_RR_2
            ctypes.POINTER(real_T), ctypes.POINTER(real_T), ctypes.POINTER(real_T),  # &arg_x, &arg_y, &arg_phi
            ctypes.POINTER(real_T), ctypes.POINTER(real_T), ctypes.POINTER(real_T),  # &arg_VE_vx, &arg_VE_vy, &arg_VE_r
            ctypes.POINTER(real_T), ctypes.POINTER(real_T),  # &arg_VE_ax, &arg_VE_ay
            (real_T * 4),  # arg_Fz
            ctypes.POINTER(real_T), ctypes.POINTER(real_T), ctypes.POINTER(real_T),  # &arg_steering_rad, &arg_steering_rate, &arg_steering_torque
            (real_T * 4), (real_T * 4), (real_T * 4),  # arg_wheel_speed, arg_wheel_speed_max, arg_wheel_speed_min
            (real_T * 4), (real_T * 4),  # arg_wheel_torque, arg_wheel_torque_target
            ctypes.POINTER(real_T), ctypes.POINTER(real_T), ctypes.POINTER(real_T),  # &arg_DV_DC, &arg_DV_delta, &arg_yaw_moment_target
        ]

        self.step_func.restype = None 


    def update_dynamics(self, dt, input_command):
        
        throttle_input = input_command['throttle']
        steering_input = input_command['steering']

        for _ in range(int(dt/0.001)):
            self.step_func(self.car_model_instance,
                    real_T(0), real_T(0),  # example values for the first two parameters
                    real_T(throttle_input), real_T(steering_input),  # example values for the first two parameters
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0), 
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0),
                    ctypes.byref(self.arg_x), ctypes.byref(self.arg_y), ctypes.byref(self.arg_phi),
                    ctypes.byref(self.arg_VE_vx), ctypes.byref(self.arg_VE_vy), ctypes.byref(self.arg_VE_r),
                    ctypes.byref(self.arg_VE_ax), ctypes.byref(self.arg_VE_ay),
                    (real_T * 4)(10.0, 10.0, 10.0, 10.0),
                    ctypes.byref(self.arg_steering_rad), ctypes.byref(self.arg_steering_rate), ctypes.byref(self.arg_steering_torque),
                    (real_T * 4)(0.0, 0.0, 0.0, 0.0), (real_T * 4)(0.0, 0.0, 0.0, 0.0), (real_T * 4)(0.0, 0.0, 0.0, 0.0),
                    (real_T * 4)(0.0, 0.0, 0.0, 0.0), (real_T * 4)(0.0, 0.0, 0.0, 0.0), (real_T * 4)(0.0, 0.0, 0.0, 0.0),
                    ctypes.byref(self.arg_DV_DC), ctypes.byref(self.arg_DV_delta), ctypes.byref(self.arg_yaw_moment_target)
                    )

    def get_state(self):
        self.state['x'] = self.arg_x.value
        self.state['y'] = self.arg_y.value
        self.state['yaw'] = self.arg_phi.value
        self.state['v_x'] = self.arg_VE_vx.value
        self.state['v_y'] = self.arg_VE_vy.value
        self.state['r'] = self.arg_VE_r.value
        self.state['a_x'] = self.arg_VE_ax.value
        self.state['a_y'] = self.arg_VE_ay.value
        self.state['steer'] = self.arg_steering_rad.value

        return self.state
    
    def Destructor(self):
        self.CarModel_delete(self.car_model_instance)

def main():
    vehicle = Vehicle()
    dt = 0.1  # Time step for the simulation

    try:
        while True:
            # Example command, replace with real command inputs
            input_command = {'throttle': 1.0, 'steering': 0.1}
            
            vehicle.update_dynamics(dt, input_command)
            print(vehicle.get_state())
            time.sleep(dt)
            
    except KeyboardInterrupt:
        print("Simulation ended.")
        Vehicle.Destructor(vehicle)


In [31]:
main()

{'x': 0.01710610237945212, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.18766472261505546, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.1716483872552705, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.03671947360756918, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.20474759734201525, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.17009280127000637, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.05803345379660028, 'y': 0.0, 'yaw': 0.0, 'v_x': 0.22167557065653626, 'v_y': 0.0, 'r': 0.0, 'a_x': 0.16854995339659246, 'a_y': 0.0, 'steer': 0.0}
{'x': 0.0810252107543257, 'y': 6.506513703467037e-05, 'yaw': 7.673201178535973e-05, 'v_x': 0.23813535867541594, 'v_y': 0.0058765543484778394, 'r': 0.007457060171376277, 'a_x': 0.15793119136650377, 'a_y': 0.1382419768508512, 'steer': 0.05052492611971217}
{'x': 0.10563181566550865, 'y': 0.0010481584988245069, 'yaw': 0.0013302148182135203, 'v_x': 0.25428960202313566, 'v_y': 0.011727592504944321, 'r': 0.015275236048036074, 'a_x': 0.16403606481140734, 'a_y': 0.02444373229016063, 'steer': 0.09189946909062566}
{'x': 0.13187254384184505, 'y': 0

In [14]:
import ctypes
import os
import time
# Load the shared library
# Get the current directory where this script and your libraries are located

current_dir = os.getcwd()

current_ld_library_path = os.environ.get('LD_LIBRARY_PATH', '')
new_ld_library_path = os.path.abspath('.') + ':' + current_ld_library_path
os.environ['LD_LIBRARY_PATH'] = new_ld_library_path

# Construct the full path to the shared library
libvehicle_path = os.path.join(current_dir, 'libvehicle_and_lowlevel_control.so')

lib = ctypes.CDLL(libvehicle_path)

# Create an instance of CarModel
CarModel_new = lib.CarModel_new
CarModel_new.restype = ctypes.c_void_p

CarModel_initialize = lib.CarModel_initialize
CarModel_initialize.argtypes = [ctypes.c_void_p]

CarModel_delete = lib.CarModel_delete
CarModel_delete.argtypes = [ctypes.c_void_p]

real_T = ctypes.c_double

step_func = lib.CarModel_step
step_func.argtypes = [
    ctypes.c_void_p,  # instance pointer
    real_T, real_T,  # tuning_param_key, tuning_param_value
    real_T, real_T,  # Command_throttle, Command_steering
    real_T, real_T, real_T,  # a_x_target0, a_x_target1, a_x_target2
    real_T, real_T, real_T,  # delta_target0, delta_target1, delta_target2
    real_T, real_T, real_T,  # yawrate_target0, yawrate_target1, yawrate_target2
    real_T, real_T, real_T,  # vx_mpc0, vx_mpc1, vx_mpc2
    real_T, real_T, real_T,  # vy_mpc0, vy_mpc1, vy_mpc2
    real_T, real_T, real_T,  # F_x_target_FL_0, F_x_target_FL_1, F_x_target_FL_2
    real_T, real_T, real_T,  # F_x_target_FR_0, F_x_target_FR_1, F_x_target_FR_2
    real_T, real_T, real_T,  # F_x_target_RL_0, F_x_target_RL_1, F_x_target_RL_2
    real_T, real_T, real_T,  # F_x_target_RR_0, F_x_target_RR_1, F_x_target_RR_2
    ctypes.POINTER(real_T), ctypes.POINTER(real_T), ctypes.POINTER(real_T),  # &arg_x, &arg_y, &arg_phi
    ctypes.POINTER(real_T), ctypes.POINTER(real_T), ctypes.POINTER(real_T),  # &arg_VE_vx, &arg_VE_vy, &arg_VE_r
    ctypes.POINTER(real_T), ctypes.POINTER(real_T),  # &arg_VE_ax, &arg_VE_ay
    (real_T * 4),  # arg_Fz
    ctypes.POINTER(real_T), ctypes.POINTER(real_T), ctypes.POINTER(real_T),  # &arg_steering_rad, &arg_steering_rate, &arg_steering_torque
    (real_T * 4), (real_T * 4), (real_T * 4),  # arg_wheel_speed, arg_wheel_speed_max, arg_wheel_speed_min
    (real_T * 4), (real_T * 4),  # arg_wheel_torque, arg_wheel_torque_target
    ctypes.POINTER(real_T), ctypes.POINTER(real_T), ctypes.POINTER(real_T),  # &arg_DV_DC, &arg_DV_delta, &arg_yaw_moment_target
]

step_func.restype = None

# Usage
car_model_instance = CarModel_new()
CarModel_initialize(car_model_instance)

# SETTING THE INPUTS TO THE VEHICLE MODEL
arg_x = real_T()
arg_y = real_T()
arg_phi = real_T()

arg_VE_vx, arg_VE_vy, arg_VE_r  = real_T(), real_T(), real_T()
arg_VE_ax, arg_VE_ay = real_T(), real_T()

arg_steering_rad, arg_steering_rate, arg_steering_torque = real_T(), real_T(), real_T()

arg_DV_DC, arg_DV_delta, arg_yaw_moment_target = real_T(), real_T(), real_T()

dt = 0.5
try:
    while True:
        for _ in range(int(dt/0.001)):
            step_func(car_model_instance,
                    real_T(0), real_T(0),  # example values for the first two parameters
                    real_T(1), real_T(0),  # example values for the first two parameters
                    real_T(0), real_T(0),real_T(0),
                    real_T(0.1), real_T(0.1),real_T(0.1),
                    real_T(0), real_T(0),real_T(0), 
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0),
                    real_T(0), real_T(0),real_T(0),
                    ctypes.byref(arg_x), ctypes.byref(arg_y), ctypes.byref(arg_phi),
                    ctypes.byref(arg_VE_vx), ctypes.byref(arg_VE_vy), ctypes.byref(arg_VE_r),
                    ctypes.byref(arg_VE_ax), ctypes.byref(arg_VE_ay),
                    (real_T * 4)(0.0, 0.0, 0.0, 0.0),
                    ctypes.byref(arg_steering_rad), ctypes.byref(arg_steering_rate), ctypes.byref(arg_steering_torque),
                    (real_T * 4)(0.0, 0.0, 0.0, 0.0), (real_T * 4)(0.0, 0.0, 0.0, 0.0), (real_T * 4)(0.0, 0.0, 0.0, 0.0),
                    (real_T * 4)(0.0, 0.0, 0.0, 0.0), (real_T * 4)(0.0, 0.0, 0.0, 0.0), (real_T * 4)(0.0, 0.0, 0.0, 0.0),
                    ctypes.byref(arg_DV_DC), ctypes.byref(arg_DV_delta), ctypes.byref(arg_yaw_moment_target)
                    )

        # Access and print the updated values
        print("x:", arg_x.value, "y:", arg_y.value, "vx:", arg_VE_vx.value, "vy:", arg_VE_vy.value)
        time.sleep(dt)
        
except KeyboardInterrupt:
    print("Simulation ended.")

# When done, delete the instance to avoid memory leaks
CarModel_delete(car_model_instance)


x: 0.10563181566550865 y: 0.0010481584988245069 vx: 0.25428960202313566 vy: 0.011727592504944321
x: 0.25308768550500893 y: 0.009012369948841536 vx: 0.33532198910080324 vy: 0.016151571028187418
x: 0.44005602693905727 y: 0.02109063449385016 vx: 0.41281427657168374 vy: 0.0198360710714521
x: 0.6646587646418216 y: 0.03849166882494097 vx: 0.4868432920995783 vy: 0.023339665608634867
x: 0.9250382490580917 y: 0.06262753848136612 vx: 0.5575540785072577 vy: 0.026671651156720214
Simulation ended.


0